In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
# Use Spark to read in the 'customer_churn.csv' file.
data = spark.read.csv('customer_churn.csv', header = True, inferSchema = True)

In [ ]:
# Prin the Schema of the DataFrame
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [ ]:
data.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

In [ ]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [ ]:
data.show(3)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 3 rows



In [ ]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [ ]:
data_pre = assembler.transform(data)

In [ ]:
data_pre.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|            features|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|[42.0,11066.8,0.0...|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|[41.0,11916.22,0....|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|[38.0,12884.75,0....|
|   Phillip White|42.0|       8010.76|  

In [ ]:
final_data = data_pre.select('features', 'churn')

In [ ]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- churn: integer (nullable = true)



In [ ]:
final_data.show(3)

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
+--------------------+-----+
only showing top 3 rows



In [ ]:
final_data = data_pre.select('features', 'churn')
final_data.count()

900

In [ ]:
final_data = final_data.na.drop()
final_data.count()

900

In [ ]:
# split the data into train and test
train_churn,test_churn = final_data.randomSplit([0.8, 0.2])

In [ ]:
# build the model
# Create a Logistic Regression Model object
logistic = LogisticRegression(featuresCol='features',
                      labelCol='churn',
                      predictionCol='prediction')

In [ ]:
# Fit the model to the data and call this model logisticModel
logisticModel = logistic.fit(train_churn,)

In [ ]:
training_sum = logisticModel.summary

In [ ]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                721|                721|
|   mean|0.17059639389736478|0.13314840499306518|
| stddev| 0.3764170329690199|0.33997089973642053|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [ ]:
# Create a predictions for the testing data and show confusion matrix
test_model = logisticModel.transform(test_churn)
test_model.groupBy('churn', 'prediction').count().show()

+-----+----------+-----+
|churn|prediction|count|
+-----+----------+-----+
|    1|       0.0|   13|
|    0|       0.0|  147|
|    1|       1.0|   14|
|    0|       1.0|    5|
+-----+----------+-----+



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [ ]:
# Calculate the elements of the confusion matrix
TN = test_model.filter('prediction = 0 and churn = prediction').count()
TP = test_model.filter('prediction = 1 and churn = prediction').count()
FN = test_model.filter('prediction = 0 and churn != prediction').count()
FP = test_model.filter('prediction = 1 and churn != prediction').count()

In [ ]:
# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall   = {:.2}'.format(precision, recall))

precision = 0.74
recall   = 0.52


In [ ]:
print('Accuracy: ', (TN + TP) / (TN+TP+FN+FP))

Accuracy:  0.8994413407821229


In [ ]:
# evaluate the results
pred_and_labels = logisticModel.evaluate(test_churn)

In [ ]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[29.0,9617.59,0.0...|    0|[4.45434615285144...|[0.98850573413494...|       0.0|
|[29.0,13240.01,1....|    0|[6.67289134229972...|[0.99873686086562...|       0.0|
|[29.0,13255.05,1....|    0|[4.02376220764521...|[0.98242872288541...|       0.0|
|[30.0,10960.52,1....|    0|[2.24712308963458...|[0.90440208967169...|       0.0|
|[31.0,7073.61,0.0...|    0|[3.04009837396205...|[0.95435311489631...|       0.0|
|[31.0,9574.89,0.0...|    0|[3.45385224570171...|[0.96934581541338...|       0.0|
|[31.0,12264.68,1....|    0|[3.46184874262364...|[0.96958253765662...|       0.0|
|[32.0,8575.71,0.0...|    0|[3.68225162532962...|[0.97545154638725...|       0.0|
|[32.0,9036.27,0.0...|    0|[-0.2322300625783...|[0.44220200884041...|       1.0|
|[32.0,11715.72,

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [ ]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [ ]:
auc

0.7428118908382066

## Predict on new data

In [ ]:
final_lr_model = logistic.fit(final_data)

In [ ]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [ ]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [ ]:
test_new_customers = assembler.transform(new_customers)

In [ ]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
final_results= final_lr_model.transform(test_new_customers)

In [ ]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [ ]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          